# Estimating distribution parameters

>If I were to write a sampler for it, I'd first write one where I knew the correct ordering and the stage the participant's disease was in and inferring the means of the Gaussians for both distributions based on that. We didn't discuss priors on the Gaussians means and variances, but I would use conjugate ones in which case the update is exact.

Therefore, in this estimation, I assume that we know $S$ (therefore $S_n$) and $k_j$ and am estimating $\theta$ and $\phi$ based on participants' data. We also know that for each biomarker, its value is drawn from a normal distribution, either from the healthy ($\theta$) or the diseased ($\phi$) distribution.

## Conjugate priors

Conjugacy occurs when the posterior distribution is in the same family of distribution as the prior distribution, but with new parameter values. 

Why conjugacy is important? Because without it, one has to do the integration, which oftentimes is hard. 

Three major conjugate families:

- Beta-Binomial
- Gamma-Poisson
- Normal-Normal

In our example, we assume that the measurement data for each biomarker follows a normal distribution; however, we do not know the exact $\mu$ and $\sigma$. Our job is to estimate the two parameters for each biomarker based on the data we have. 

According to [*An Introduction to Bayesian Thinking*](https://statswithr.github.io/book/inference-and-decision-making-with-multiple-parameters.html#sec:normal-gamma) by Clyde et al. (2022), if the data comes from a normal distribution with unknown $\mu$ and $\sigma$, the conjugate prior for $\mu$ has a normal distribution with mean $m_0$ and variance $\frac{\sigma^2}{n_0}$. The conjugate prior for $\frac{1}{\sigma^2}$ has a Gamma distribution with shape $\frac{v_0}{2}$ and rate $\frac{v_0 s_0^{2}}{2}$ where

- $m_0$: prior estimate of $\mu$.
- $n_0$: how strongly is the prior belief in $m_0$ is held.
- $s_0^2$: prior estimate of $\sigma^2$.
- $v_0$: prior degress of freedome, influencing the certainty of $s_0^2$.

That is to say: 

$$\mu | \sigma^2 \sim \mathcal{N}(m_0, \sigma^2/n_0)$$

$$1/\sigma^2 \sim Gamma\left(\frac{v_0}{2}, \frac{v_0 s_0^2}{2} \right)$$

Combined, we have:

$$(\mu, 1/\sigma^2) \sim NormalGamma(m_0, n_0, s_0^2, v_0)$$

The posterior also follows a Normal-Gamma distribution:

$$(\mu, 1/\sigma^2) | data \sim NormalGamma(m_n, n_n, s_n^2, v_n)$$

More specifically

$$1/\sigma^2 | data \sim Gamma(v_n/2, s_n^2 v_n/2)$$

$$\mu | data, \sigma^2 \sim \mathcal{N}(m_n, \sigma^2/n_n)$$

Based on the above two equations, we know that the mean of posterior mean is $m_n$ and the mean of the posterior variance is $(s_n^2 v_n/2)/(v_n/2)$. This is beceause the expected value of $Gamma(\alpha, \beta)$ is $\frac{\alpha}{\beta}$.

where 

- $m_n$: posterior mean, mode, and median for $\mu$
- $n_n$: posterior sample size 
- $s_n^2$: posterior variance 
- $v_n$: posterior degrees of freedome

The updating rules to get the new hyper-parameters: 

$$m_n = \frac{n \bar{y} + n_0m_0}{n + n_0}$$

$$\frac{n}{n+n_0} \bar{y} + \frac{n_0}{n+n_0}m_0$$

$$n_n = n_0 + n$$

$$v_n = v_0 + n$$

$$s_n^2 = \frac{1}{v_n}\left[s^2(n-1) + s_0^2v_0 + \frac{n_0n}{n_n}(\bar{y}-m_0)^2\right]$$

where

- $n$: sample size
- $\bar{y}$: sample mean 
- $s^2$: sample variance

## Implementing estimation

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import altair as alt

In [2]:
def estimate_params_exact(m0, n0, s0_sq, v0, data):
    '''This is to estimate means and vars based on conjugate priors
    Inputs:
        - data: a vector of measurements 
        - m0: prior estimate of $\mu$.
        - n0: how strongly is the prior belief in $m_0$ is held.
        - s0_sq: prior estimate of $\sigma^2$.
        - v0: prior degress of freedome, influencing the certainty of $s_0^2$.
    
    Outputs:
        - mu estiate, std estimate
    '''
    # Data summary
    sample_mean = np.mean(data)
    sample_size = len(data)
    sample_var = np.var(data, ddof=1)  # ddof=1 for unbiased estimator

    # Update hyperparameters for the Normal-Inverse Gamma posterior
    updated_m0 = (n0 * m0 + sample_size * sample_mean) / (n0 + sample_size)
    updated_n0 = n0 + sample_size
    updated_v0 = v0 + sample_size 
    updated_s0_sq = (1 / updated_v0) * ((sample_size - 1) * sample_var + v0 * s0_sq + 
                    (n0 * sample_size / updated_n0) * (sample_mean - m0)**2)
    updated_alpha = updated_v0/2
    updated_beta = updated_v0*updated_s0_sq/2

    # Posterior estimates
    mu_posterior_mean = updated_m0
    sigma_squared_posterior_mean = updated_beta/updated_alpha

    mu_estimation = mu_posterior_mean
    std_estimation = np.sqrt(sigma_squared_posterior_mean)

    return mu_estimation, std_estimation

In [3]:
# truth data; used to validate the estimates
truth = pd.read_csv('data/means_stds.csv')
# observed data:
observed_data = pd.read_csv('data/participant_data.csv')
observed_data.head()

,Biomarker,participant,measurement,k_j,S_n,affected_or_not
0,Biomarker 0,0,28.430422,5,1,affected
1,Biomarker 0,1,27.271975,4,1,affected
2,Biomarker 0,2,15.562816,0,1,not_affected
3,Biomarker 0,3,27.592558,5,1,affected
4,Biomarker 0,4,28.282987,4,1,affected


In [4]:
biomarkers = [_ for _ in range(10)]

In [5]:
def get_estimated_means_stds_df(observed_data):
    '''This is to get the dataframe of estimated means and stds 
    Inputs:
        - observed_data: participants data 
    Outputs:
        - a pandas dataframe containing theta and phi means and stds
    '''
    # empty list of dictionaries to store the estimates 
    means_stds_estimate_dict_list = []
    
    for biomarker in biomarkers: 
        dic = {'biomarker': biomarker}  # Initialize dictionary outside the inner loop
        for i in ['affected', 'not_affected']:
            data_full = observed_data[(observed_data.Biomarker == f"Biomarker {biomarker}") & (
            observed_data.affected_or_not == i)]
            data = data_full.measurement
            mu_estimate, std_estimate = estimate_params_exact(
                m0 = 0, n0 = 1, s0_sq = 1, v0 = 1, data=data)
            if i == 'affected':
                dic['theta_mean'] = mu_estimate
                dic['theta_std'] = std_estimate
            else:
                dic['phi_mean'] = mu_estimate
                dic['phi_std'] = std_estimate
        print(f"biomarker {biomarker} done!")
        means_stds_estimate_dict_list.append(dic)
    estimate_means_stds_df = pd.DataFrame(means_stds_estimate_dict_list)
    return estimate_means_stds_df 

In [6]:
estimate_df = get_estimated_means_stds_df(observed_data)
estimate_df.to_csv("data/estimate_means_stds.csv", index = False)
estimate_df

biomarker 0 done!
biomarker 1 done!
biomarker 2 done!
biomarker 3 done!
biomarker 4 done!
biomarker 5 done!
biomarker 6 done!
biomarker 7 done!
biomarker 8 done!
biomarker 9 done!


,biomarker,theta_mean,theta_std,phi_mean,phi_std
0,0,27.736569,2.912793,16.127597,7.454735
1,1,-5.963448,1.512621,-19.916965,5.052305
2,2,243.824288,59.375090,139.352302,36.500046
3,3,-27.482929,15.060550,-40.750651,31.227442
4,4,4.470661,1.857089,-5.207810,7.329156
5,5,0.412543,0.284192,0.264135,0.319591
6,6,43.329157,17.720233,19.779650,6.689862
7,7,11.434276,4.485240,5.183918,3.560668
8,8,0.590799,0.151114,0.464785,0.216008
9,9,-10.966834,4.739591,-19.756023,6.330043


In [7]:
def obtain_theta_phi_params(biomarker, estimate_df, truth):
    '''This is to obtain both true and estimated theta and phi params for each biomarker '''
    biomarker_data_est = estimate_df[estimate_df.biomarker == biomarker].reset_index()
    biomarker_data = truth[truth.biomarker == biomarker].reset_index()
    # theta for affected
    theta_mean_est = biomarker_data_est.theta_mean[0]
    theta_std_est = biomarker_data_est.theta_std[0]

    theta_mean = biomarker_data.theta_mean[0]
    theta_std = biomarker_data.theta_std[0]

    # phi for not affected
    phi_mean_est = biomarker_data_est.phi_mean[0]
    phi_std_est = biomarker_data_est.phi_std[0]

    phi_mean = biomarker_data.phi_mean[0]
    phi_std = biomarker_data.phi_std[0]

    return theta_mean, theta_std, theta_mean_est, theta_std_est, phi_mean, phi_std, phi_mean_est, phi_std_est

In [8]:
charts = []
biomarkers = [_ for _ in range(10)]
for n in biomarkers: 
    theta_mean, theta_std, theta_mean_est, theta_std_est, phi_mean, phi_std, phi_mean_est, phi_std_est = obtain_theta_phi_params(
    n, estimate_df, truth)
    mean1, std1 = theta_mean, theta_std
    mean2, std2 = theta_mean_est, theta_std_est

    # Generating points on the x axis
    x_thetas = np.linspace(min(mean1 - 3*std1, mean2 - 3*std2), 
                    max(mean1 + 3*std1, mean2 + 3*std2), 1000)

    # Creating DataFrames for each distribution
    df1 = pd.DataFrame({'x': x_thetas, 'pdf': norm.pdf(x_thetas, mean1, std1), 'Distribution': 'Actual'})
    df2 = pd.DataFrame({'x': x_thetas, 'pdf': norm.pdf(x_thetas, mean2, std2), 'Distribution': 'Estimated'})

    # Combining the DataFrames
    df3 = pd.concat([df1, df2])

    # Altair plot
    chart_theta = alt.Chart(df3).mark_line().encode(
        x='x',
        y='pdf',
        color=alt.Color('Distribution:N', legend=alt.Legend(title="Theta"))
    ).properties(
        title=f'Biomarker {n}, Theta'
        )

    mean1, std1 = phi_mean, phi_std
    mean2, std2 = phi_mean_est, phi_std_est

    # Generating points on the x axis
    x_phis = np.linspace(min(mean1 - 3*std1, mean2 - 3*std2), 
                    max(mean1 + 3*std1, mean2 + 3*std2), 1000)

    # Creating DataFrames for each distribution
    df1 = pd.DataFrame({'x': x_phis, 'pdf': norm.pdf(x_phis, mean1, std1), 'Distribution': 'Actual'})
    df2 = pd.DataFrame({'x': x_phis, 'pdf': norm.pdf(x_phis, mean2, std2), 'Distribution': 'Estimated'})

    # Combining the DataFrames
    df3 = pd.concat([df1, df2])

    # Altair plot
    chart_phi = alt.Chart(df3).mark_line().encode(
        x='x',
        y='pdf',
        color=alt.Color('Distribution:N', legend=alt.Legend(title="Phi"))
    ).properties(
        title=f'Biomarker {n}, Phi'
        )
    
    # Concatenate theta and phi charts horizontally
    hconcat_chart = alt.hconcat(chart_theta, chart_phi).resolve_scale(color="independent")

    # Append the concatenated chart to the list of charts
    charts.append(hconcat_chart)
# Concatenate all the charts vertically
final_chart = alt.vconcat(*charts)

# Display the final chart
final_chart.display()

alt.VConcatChart(...)